In [1]:
import torch

import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.preprocessing import MinMaxScaler
from pandas.plotting import register_matplotlib_converters
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
import random
from scipy.interpolate import UnivariateSpline  
import pickle
from sklearn.decomposition import PCA
import time
import datetime
from sklearn.metrics import mean_squared_error
from math import sqrt
from pathlib import Path
from pprint import pprint

from model import MV_LSTM

In [2]:
base_path = Path("../dataset")

In [3]:
from numpy import array
from numpy import hstack

# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    
    for i in range(0,len(sequences),100):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if i!=0 and end_ix > len(sequences):
            break
        
        sequences[i:end_ix,0]=np.insert(np.diff(sequences[i:end_ix,0]),0,0)
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix-33], sequences[end_ix-33:end_ix]
        
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [4]:
df = pd.read_csv(base_path/"modified_usa.csv")

is_nd = (df["State"] == "North Dakota")

df2 = df.copy(deep = True)

is_ken = (df["State"].isin(["Kentucky", "Minnesota"]))

In [6]:
data = df[df.State.isin(['Alabama', 'Alaska', 'American Samoa', 'Arizona', 'Arkansas',
       'California', 'Colorado', 'Connecticut', 'Delaware',
       'Diamond Princess', 'District of Columbia', 'Florida', 'Georgia',
       'Grand Princess', 'Guam', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Northern Mariana Islands', 'Ohio', 'Oklahoma',
       'Oregon', 'Pennsylvania', 'Puerto Rico', 'Rhode Island',
       'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah',
       'Vermont', 'Virgin Islands', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'])][["Confirmed", "Deaths", "lat", "long"]]


In [7]:
data2=df2[(is_ken)][['Confirmed','lat','long','Deaths']]
date=df2[(is_ken)][['Date','Confirmed']]

date["Date"] = pd.to_datetime(date["Date"], format = '%Y%m%d', errors = 'ignore')
date.set_index('Date', inplace = True)

In [17]:
n_features = 4 # this is number of parallel inputs
n_timesteps = 500 # this is number of timesteps

In [19]:
X, Y = split_sequences(data.values, n_timesteps)
print (X.shape,Y.shape) 

(9677, 467, 4) (9677, 33, 4)


In [20]:
alld=np.concatenate((X,Y),1)
alld=alld.reshape(alld.shape[0]*alld.shape[1],alld.shape[2])

scaler = MinMaxScaler()
scaler.fit(alld)
X=[scaler.transform(x) for x in X]
y=[scaler.transform(y) for y in Y]

X=np.array(X)
y=np.array(y)[:,:,0]

In [21]:
# mv_net = MV_LSTM(n_features,67).cuda()
mv_net = MV_LSTM(n_features,seq_length= 467, timesteps= n_timesteps).cuda()

criterion = torch.nn.MSELoss() # reduction='sum' created huge loss value
optimizer = torch.optim.Adam(mv_net.parameters(), lr=1e-3)

train_episodes = 1000

batch_size = 16
losses = []

mv_net.train()

for t in range(train_episodes):
    
    for b in range(0,len(X),batch_size):
       
        p = np.random.permutation(len(X))
        
        inpt = X[p][b:b+batch_size,:,:]
        target = y[p][b:b+batch_size,:]    
        
        x_batch = torch.tensor(inpt,dtype=torch.float32).cuda()    
        y_batch = torch.tensor(target,dtype=torch.float32).cuda()
       
        mv_net.init_hidden(x_batch.size(0))
        
        output = mv_net(x_batch) 
        
        # print(x_batch[:,:,0].shape, y_batch.shape, torch.cat((x_batch[:,:,0], y_batch), 1).shape)
        
        all_batch=torch.cat((x_batch[:,:,0], y_batch), 1)
        
        # print(output.shape, all_batch.view(-1).shape)
        
        loss = 1000*criterion(output.view(-1), all_batch.view(-1))  

        loss.backward()
        optimizer.step()        
        optimizer.zero_grad() 
        
    
    losses.append(loss.item())    
    print('step : ' , t , 'loss : ' , loss.item())

with open('../model/losses.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump(losses, f,protocol=2)

RuntimeError: size mismatch, m1: [16 x 14010], m2: [44010 x 500] at C:/w/b/windows/pytorch/aten/src\THC/generic/THCTensorMathBlas.cu:283

In [14]:
fig, ax = plt.subplots(figsize = (12, 4))
losses = pd.Series(losses)
ax.set_xlabel("train steps", fontsize = 15)
ax.set_ylabel("loss")
losses.plot(lw = 3)

TypeError: no numeric data to plot

In [ ]:
data2x=data2
truth = data2

data2x.values[0:len(data2x),0]=np.insert(np.diff(data2x.values[0:len(data2x),0]),0,0)
data2x=scaler.transform(data2x) 


X_test = np.expand_dims(data2x, axis=0)
# print (X_test.shape)
mv_net.init_hidden(1)


lstm_out = mv_net(torch.tensor(X_test[:,-67:,:],dtype=torch.float32).cuda())
lstm_out=lstm_out.reshape(1,100,1).cpu().data.numpy()

# print (data2x[-67:,0],lstm_out)
actual_predictions = scaler.inverse_transform(np.tile(lstm_out, (1, 1,4))[0])[:,0]


#actual_predictions=lstm_out


x = np.arange(0, 54, 1)
x2 = np.arange(0, 67, 1)
x3 = np.arange(0, 100, 10)
x4 = np.arange(0, 50, 1)


#save prediction
with open('../predictions/predict_ken.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump(pd.Series(actual_predictions), f,protocol=2)

In [58]:
fig, ax = plt.subplots() 
plt.title('Days vs Confirmed Cases Accumulation')
plt.ylabel('Confirmed')

left, width = .25, .5
bottom, height = .25, .5
right = left + width
top = bottom + height

print (date.index)
date_list=pd.date_range(start=date.index[0],end=date.index[-1])
print (date_list)

plt.axvline(x=np.array(date_list)[66], color='r', linestyle='--')

ax.text(0.2*(left+right), 0.8*(bottom+top), 'input sequence',
        horizontalalignment='left',
        verticalalignment='center',
        fontsize=10, color='red',
        transform=ax.transAxes)
ax.text(0.0125*(left+right), 0.77*(bottom+top), '______________________',
        horizontalalignment='left',
        verticalalignment='center',
        fontsize=20, color='red',
        transform=ax.transAxes)



sumpred=np.cumsum(np.absolute(actual_predictions))

print(date.Confirmed.shape, sumpred.shape)
print (date.values.shape) 
print (sqrt(mean_squared_error(date.Confirmed,sumpred)))          
plt.plot(date.values[-67:],np.cumsum(data2.confirmed.values[-67:]))
plt.plot(np.array(date_list),sumpred,label='Prediction')
plt.plot(np.array(date_list),date.confirmed,label='Actual')
plt.xticks(rotation=90)
fig.autofmt_xdate()
plt.legend(loc=2)
plt.show() 

Index(['2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25', '2020-01-26',
       '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30', '2020-01-31',
       ...
       '2020-11-17', '2020-11-18', '2020-11-19', '2020-11-20', '2020-11-21',
       '2020-11-22', '2020-11-23', '2020-11-24', '2020-11-25', '2020-11-26'],
      dtype='object', name='Date', length=37820)
DatetimeIndex(['2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25',
               '2020-01-26', '2020-01-27', '2020-01-28', '2020-01-29',
               '2020-01-30', '2020-01-31',
               ...
               '2020-11-17', '2020-11-18', '2020-11-19', '2020-11-20',
               '2020-11-21', '2020-11-22', '2020-11-23', '2020-11-24',
               '2020-11-25', '2020-11-26'],
              dtype='datetime64[ns]', length=310, freq='D')
(37820,) (100,)
(37820, 1)


ValueError: Found input variables with inconsistent numbers of samples: [37820, 100]